<a href="https://colab.research.google.com/github/vicknentura/ML-in-SQL-BigQuery/blob/main/Google_Cloud_x_MLB(TM)_Hackathon_Exploring_MLB_Provided_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Import Python Libraries
# General data science libraries
import pandas as pd
import numpy as np

# Pulling data from APIs, parsing JSON
import requests
import json

# Interfacing w/ Cloud Storage from Python
from google.cloud import storage

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# ML Evaluation
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder

In [2]:
#@title Modify Settings

# Expand max column width when displaying data frames to handle longer text
pd.set_option('display.max_colwidth', 200)

In [3]:
#@title Function to Load Newline Delimited JSON into Pandas DF
def load_newline_delimited_json(url):
    """Loads a newline-delimited JSON file from a URL into a pandas DataFrame.

    Args:
        url: The URL of the newline-delimited JSON file.

    Returns:
        A pandas DataFrame containing the data, or None if an error occurs.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        data = []
        for line in response.text.strip().split('\n'):
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Skipping invalid JSON line: {line} due to error: {e}")

        return pd.DataFrame(data)
    except requests.exceptions.RequestException as e:
        print(f"Error downloading data: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

In [4]:
#@title Function to Process Results from Various MLB Stats API Endpoints
def process_endpoint_url(endpoint_url, pop_key=None):
  """
  Fetches data from a URL, parses JSON, and optionally pops a key.

  Args:
    endpoint_url: The URL to fetch data from.
    pop_key: The key to pop from the JSON data (optional, defaults to None).

  Returns:
    A pandas DataFrame containing the processed data
  """
  json_result = requests.get(endpoint_url).content

  data = json.loads(json_result)

   # if pop_key is provided, pop key and normalize nested fields
  if pop_key:
    df_result = pd.json_normalize(data.pop(pop_key), sep = '_')
  # if pop_key is not provided, normalize entire json
  else:
    df_result = pd.json_normalize(data)

  return df_result

# **Methodology 📓**

1. Train the regression-based prediction model using the MLB statistics on each player from each team.
  * Test linear regression, LASSO, Ridge, and ensemble methods like DT and RF

2. Find an optimal set of player stats using available performance fields and evaluate model performance using regression metrics (MSE, RMSE, and MAE)
  * Consider feature selection techniques like RFE and feature importnace in the case of DT and RF

3. Use historical league data as layers of training to reinforce the best player rankings potentially using a neural network and re-evaluate the performance
  * Transfer learning

4. Run the players from other leagues through the model and obtain a ranking
  * Consider a validation step for how well the model performs on these players compared to MLB players

## **Pre-processing and Data Exploration 🧗**

### **MLB League Dataset**

In [5]:
# Define the endpoint URL for the specific league (MLB)
league_endpoint_url = 'https://statsapi.mlb.com/api/v1/league'

# Function to process the endpoint URL
def process_endpoint_url(url):
    response = requests.get(url)
    return response.json()

# Fetch league data
league_data = process_endpoint_url(league_endpoint_url)

# Extract leagues data
leagues = league_data['leagues']

# Convert leagues data to a DataFrame
leagues_df = pd.DataFrame(leagues)

# Drop the 'copyright' column if it exists
leagues_df = leagues_df.drop(columns=['copyright'], errors='ignore')

# Check if 'seasonDateInfo' exists and is a dictionary
if 'seasonDateInfo' in leagues_df.columns:
    # Normalize the 'seasonDateInfo' field
    season_date_info_df = pd.json_normalize(leagues_df['seasonDateInfo'])

    # Combine the original leagues DataFrame with the new season date info DataFrame
    leagues_df = pd.concat([leagues_df.drop(columns=['seasonDateInfo']), season_date_info_df], axis=1)


def prefix_columns(df, prefix):
    new_columns = {col: f"{prefix}_{col}" for col in df.columns}
    return df.rename(columns=new_columns)

leagues_df = prefix_columns(leagues_df, "league")

# Display the DataFrame
print(leagues_df)

     league_id             league_name         league_link  \
0          103         American League  /api/v1/league/103   
1          104         National League  /api/v1/league/104   
2          114           Cactus League  /api/v1/league/114   
3          115       Grapefruit League  /api/v1/league/115   
4          117    International League  /api/v1/league/117   
..         ...                     ...                 ...   
111        107        College Baseball  /api/v1/league/107   
112        108        College Baseball  /api/v1/league/108   
113        587          Showcase Games  /api/v1/league/587   
114        625           Tournament 12  /api/v1/league/625   
115        577  National Pro Fastpitch  /api/v1/league/577   

    league_abbreviation  league_nameShort league_seasonState  \
0                    AL          American          preseason   
1                    NL          National          preseason   
2                    CL            Cactus          preseason   

### **MLB Team Datasets**

In [6]:
# Define the URL for the teams endpoint
team_endpoint_url = 'https://statsapi.mlb.com/api/v1/teams/'

# Function to process the endpoint URL
def process_endpoint_url(url):
    response = requests.get(url)
    return response.json()

# Fetch team data
team_data = process_endpoint_url(team_endpoint_url)

# Extract teams data
teams = team_data['teams']

# Convert teams data to a DataFrame
teams_df = pd.DataFrame(teams)

# Drop the 'copyright' and 'division' columns
columns_to_drop = ['copyright', 'division']

for column in columns_to_drop:
    if column in teams_df.columns:
        teams_df = teams_df.drop(columns=[column])  # Or teams_df.drop(column, axis=1)

# Normalize the 'venue' field
if 'venue' in teams_df.columns:
    if isinstance(teams_df['venue'].iloc[0], dict):  # Check if the first element is a dictionary
        venue_df = pd.json_normalize(teams_df['venue'])
        # Append a prefix to the columns
        venue_df.columns = [f'venue_{col}' for col in venue_df.columns]
        teams_df = pd.concat([teams_df.drop(columns=['venue']), venue_df], axis=1)

# Normalize the 'sport' field
if 'sport' in teams_df.columns:
    if isinstance(teams_df['sport'].iloc[0], dict):
        sport_df = pd.json_normalize(teams_df['sport'])
        # Append a prefix to the columns
        sport_df.columns = [f'sport_{col}' for col in sport_df.columns]
        teams_df = pd.concat([teams_df.drop(columns=['sport']), sport_df], axis=1)

# Normalize the 'league' field
if 'league' in teams_df.columns:
    if isinstance(teams_df['league'].iloc[0], dict):  # Check if the first element is a dictionary
        league_df = pd.json_normalize(teams_df['league'])
        # Append a prefix to the columns
        league_df.columns = [f'league_{col}' for col in league_df.columns]
        teams_df = pd.concat([teams_df.drop(columns=['league']), league_df], axis=1)

# Function to prefix columns
def prefix_columns(df, prefix):
    df.columns = [f'{prefix}_{col}' for col in df.columns]
    return df

# Prefix the columns of teams_df
teams_df = prefix_columns(teams_df, "teams")

# Display the teams DataFrame
print("\nTeams DataFrame:")
print(teams_df)


Teams DataFrame:
    teams_allStarStatus  teams_id                     teams_name  \
0                     N      4104  Coastal Carolina Chanticleers   
1                     N      4124          Pensacola Blue Wahoos   
2                     N      3110             Washington Huskies   
3                     N      3111           Oregon State Beavers   
4                     N      4144            Ohio State Buckeyes   
..                  ...       ...                            ...   
759                   N      5094                          North   
760                   N      5097                     SouthSouth   
761                   N      5105                           East   
762                   N      5106                           West   
763                   N      4087          Senadores de San Juan   

             teams_link  teams_season teams_teamCode teams_fileCode  \
0    /api/v1/teams/4104          2025            ccu          t4104   
1    /api/v1/teams/4124

In [7]:
print(teams_df.info())
print(teams_df['teams_league_id'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 764 entries, 0 to 763
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   teams_allStarStatus    764 non-null    object 
 1   teams_id               764 non-null    int64  
 2   teams_name             764 non-null    object 
 3   teams_link             764 non-null    object 
 4   teams_season           764 non-null    int64  
 5   teams_teamCode         764 non-null    object 
 6   teams_fileCode         763 non-null    object 
 7   teams_abbreviation     764 non-null    object 
 8   teams_teamName         764 non-null    object 
 9   teams_locationName     757 non-null    object 
 10  teams_firstYearOfPlay  751 non-null    object 
 11  teams_shortName        764 non-null    object 
 12  teams_parentOrgName    651 non-null    object 
 13  teams_parentOrgId      651 non-null    float64
 14  teams_franchiseName    733 non-null    object 
 15  teams_

### **MLB Roster Datasets**

In [8]:
# Define the endpoint URL for people (MLB)
test_endpoint_url = 'https://statsapi.mlb.com/api/v1/teams/3110/roster?season=2024'

# Function to process the endpoint URL
def process_endpoint_url(url):
    response = requests.get(url)
    return response.json()

# Fetch people data
test_data = process_endpoint_url(test_endpoint_url)

# Print the full response for inspection
print("Full People Data Response:")
print(test_data)

Full People Data Response:
{'copyright': 'Copyright 2025 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt', 'roster': [{'person': {'id': 804109, 'fullName': 'Aiva Arquette', 'link': '/api/v1/people/804109'}, 'jerseyNumber': '13', 'position': {'code': '6', 'name': 'Shortstop', 'type': 'Infielder', 'abbreviation': 'SS'}, 'status': {'code': 'A', 'description': 'Active'}}, {'person': {'id': 822704, 'fullName': 'Colin Blanchard', 'link': '/api/v1/people/822704'}, 'jerseyNumber': '23', 'position': {'code': '2', 'name': 'Catcher', 'type': 'Catcher', 'abbreviation': 'C'}, 'status': {'code': 'A', 'description': 'Active'}}, {'person': {'id': 805281, 'fullName': 'Colton Bower', 'link': '/api/v1/people/805281'}, 'jerseyNumber': '2', 'position': {'code': '2', 'name': 'Catcher', 'type': 'Catcher', 'abbreviation': 'C'}, 'status': {'code': 'A', 'description': 'Active'}}, {'person': {'id': 822596, 'full

In [9]:
# Step 1: Join leagues_df and teams_df on the 'league' field
merged_df = pd.merge(teams_df, leagues_df, left_on='teams_league_id', right_on='league_id', how='left')

# Display the merged DataFrame
print("Merged DataFrame:")
print(merged_df)

Merged DataFrame:
    teams_allStarStatus  teams_id                     teams_name  \
0                     N      4104  Coastal Carolina Chanticleers   
1                     N      4124          Pensacola Blue Wahoos   
2                     N      3110             Washington Huskies   
3                     N      3111           Oregon State Beavers   
4                     N      4144            Ohio State Buckeyes   
..                  ...       ...                            ...   
759                   N      5094                          North   
760                   N      5097                     SouthSouth   
761                   N      5105                           East   
762                   N      5106                           West   
763                   N      4087          Senadores de San Juan   

             teams_link  teams_season teams_teamCode teams_fileCode  \
0    /api/v1/teams/4104          2025            ccu          t4104   
1    /api/v1/teams/4124

In [10]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 764 entries, 0 to 763
Data columns (total 67 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   teams_allStarStatus               764 non-null    object 
 1   teams_id                          764 non-null    int64  
 2   teams_name                        764 non-null    object 
 3   teams_link                        764 non-null    object 
 4   teams_season                      764 non-null    int64  
 5   teams_teamCode                    764 non-null    object 
 6   teams_fileCode                    763 non-null    object 
 7   teams_abbreviation                764 non-null    object 
 8   teams_teamName                    764 non-null    object 
 9   teams_locationName                757 non-null    object 
 10  teams_firstYearOfPlay             751 non-null    object 
 11  teams_shortName                   764 non-null    object 
 12  teams_pa

### **MLB Historical Roster Datasets**

In [11]:
import concurrent.futures

# Function to fetch roster data for a specific team and season
def fetch_roster_data(team_id, season):
    roster_url = f'{base_url}/api/v1/teams/{team_id}/roster?season={season}'
    return process_endpoint_url(roster_url)

# Step 2: Fetch roster data for each team for multiple seasons
season_dataframes = {}  # Dictionary to hold merged DataFrames for each season

# Define the seasons
seasons = [2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013]

# Iterate through each season
for season in seasons:
    roster_data_list = []

    # Use ThreadPoolExecutor to fetch data in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_team = {executor.submit(fetch_roster_data, row['teams_id'], season): row for index, row in merged_df.iterrows()}

        for future in concurrent.futures.as_completed(future_to_team):
            team_row = future_to_team[future]
            try:
                roster_data = future.result()
                if 'roster' in roster_data:
                    roster = roster_data['roster']
                    # Append the roster data to the list with the season included
                    roster_data_list.append({
                        'team_id': team_row['teams_id'],
                        'season': season,
                        'roster': roster
                    })
            except Exception as e:
                print(f"Error fetching data for team {team_row['teams_id']} in season {season}: {e}")

    # Normalize and combine the roster data for the current season
    roster_df_list = []

    for team_roster in roster_data_list:
        team_id = team_roster['team_id']
        season = team_roster['season']
        roster = team_roster['roster']

        # Normalize the roster data
        if roster:
            roster_df = pd.json_normalize(roster)
            roster_df['team_id'] = team_id  # Add team_id to the roster DataFrame
            roster_df['season'] = season    # Add season to the roster DataFrame
            roster_df_list.append(roster_df)

    # Concatenate all roster DataFrames into a single DataFrame for the current season
    if roster_df_list:
        final_roster_df = pd.concat(roster_df_list, ignore_index=True)
    else:
        final_roster_df = pd.DataFrame()  # Empty DataFrame if no rosters were fetched

    # Merge the final roster DataFrame with the original merged_df for the current season
    current_merged_df = pd.merge(
        merged_df,
        final_roster_df,
        left_on='teams_id',  # Column from merged_df
        right_on='team_id',  # Column from final_roster_df
        how='left'           # You can change this to 'inner', 'outer', or 'right' as needed
    )

    # Store the current merged DataFrame in the dictionary
    season_dataframes[season] = current_merged_df

# Now you have separate merged DataFrames for each season
# You can access them like this:
for season, df in season_dataframes.items():
    print(f"\nMerged DataFrame for Season {season}:")
    print(df)


Merged DataFrame for Season 2024:
      teams_allStarStatus  teams_id                     teams_name  \
0                       N      4104  Coastal Carolina Chanticleers   
1                       N      4124          Pensacola Blue Wahoos   
2                       N      4124          Pensacola Blue Wahoos   
3                       N      4124          Pensacola Blue Wahoos   
4                       N      4124          Pensacola Blue Wahoos   
...                   ...       ...                            ...   
21489                   N      4087          Senadores de San Juan   
21490                   N      4087          Senadores de San Juan   
21491                   N      4087          Senadores de San Juan   
21492                   N      4087          Senadores de San Juan   
21493                   N      4087          Senadores de San Juan   

               teams_link  teams_season teams_teamCode teams_fileCode  \
0      /api/v1/teams/4104          2025            

### **MLB Individual Player Datasets**

In [15]:
# Function to fetch player data for a list of person_ids
def fetch_player_data_batch(person_ids):
    player_data_list = []
    for person_id in person_ids:
        url = f'https://statsapi.mlb.com/api/v1/people/{person_id}'
        response = requests.get(url)
        if response.status_code == 200:
            player_data = response.json()
            if 'people' in player_data and len(player_data['people']) > 0:
                player_data_list.append(player_data['people'][0])
        else:
            print(f"Error fetching data for person_id {person_id}: {response.status_code}")
    return player_data_list

# Iterate through each season's DataFrame
for season, df in season_dataframes.items():
    if 'person.link' in df.columns:
        df['person_id'] = df['person.link'].str.extract(r'/api/v1/people/(\d+)')

        # Fetch player data for each person_id in parallel
        person_ids = df['person_id'].dropna().unique()
        player_data_list = []

        # Use ThreadPoolExecutor to fetch data in parallel
        with concurrent.futures.ThreadPoolExecutor() as executor:
            # Split person_ids into chunks if necessary
            future_to_person_id = {executor.submit(fetch_player_data_batch, person_ids[i:i + 10]): i for i in range(0, len(person_ids), 10)}

            for future in concurrent.futures.as_completed(future_to_person_id):
                player_data_list.extend(future.result())

        if player_data_list:
            player_df = pd.json_normalize(player_data_list)
            player_df = player_df.rename(columns=lambda x: f'player_{x}')

            # Convert 'person_id' to numeric before merging
            df['person_id'] = pd.to_numeric(df['person_id'])
            player_df['player_id'] = pd.to_numeric(player_df['player_id'])

            df = pd.merge(
                df,
                player_df,
                left_on='person_id',
                right_on='player_id',
                how='left'
            )

            # Remove specified columns
            columns_to_remove = ['teams_id', 'teams_link', 'teams_teamCode', 'teams_fileCode',
                                 'league_id', 'league_link', 'league_orgCode',
                                 'person.link', 'position.code', 'position.abbreviation']
            df = df.drop(columns=[col for col in columns_to_remove if col in df.columns], errors='ignore')

            season_dataframes[season] = df

        print(f"\nUpdated Merged DataFrame for Season {season}:")
        print(df)
    else:
        print(f"'person_id' column not found in the DataFrame for Season {season}.")


Updated Merged DataFrame for Season 2024:
      teams_allStarStatus                     teams_name  teams_season  \
0                       N  Coastal Carolina Chanticleers          2025   
1                       N          Pensacola Blue Wahoos          2025   
2                       N          Pensacola Blue Wahoos          2025   
3                       N          Pensacola Blue Wahoos          2025   
4                       N          Pensacola Blue Wahoos          2025   
...                   ...                            ...           ...   
21495                   N          Senadores de San Juan          2024   
21496                   N          Senadores de San Juan          2024   
21497                   N          Senadores de San Juan          2024   
21498                   N          Senadores de San Juan          2024   
21499                   N          Senadores de San Juan          2024   

      teams_abbreviation teams_teamName teams_locationName  \
0     

### **Model Training and Evaluation**

In [18]:
print(filtered_data.dtypes)  # Check data types of each column
print(filtered_data.head())   # Inspect the first few rows

teams_allStarStatus          object
teams_name                   object
teams_season                  int64
teams_abbreviation           object
teams_teamName               object
                              ...  
player_nameSuffix            object
player_deathCity             object
player_deathStateProvince    object
player_deathCountry          object
allStarStatus                 int64
Length: 122, dtype: object
     teams_allStarStatus teams_name  teams_season teams_abbreviation  \
877                    N     Brazil          2022                BRA   
1697                   N      Korea          2024                KOR   
1698                   N      Korea          2024                KOR   
1699                   N      Korea          2024                KOR   
1700                   N      Korea          2024                KOR   

     teams_teamName teams_locationName teams_firstYearOfPlay teams_shortName  \
877          Brazil      United States                   NaN    

In [19]:
for col in filtered_data.select_dtypes(include=['object']).columns:
    print(f"Column: {col}")
    print(filtered_data[col].apply(type).value_counts())  # Check types in the column

Column: teams_allStarStatus
teams_allStarStatus
<class 'str'>    2922
Name: count, dtype: int64
Column: teams_name
teams_name
<class 'str'>    2922
Name: count, dtype: int64
Column: teams_abbreviation
teams_abbreviation
<class 'str'>    2922
Name: count, dtype: int64
Column: teams_teamName
teams_teamName
<class 'str'>    2922
Name: count, dtype: int64
Column: teams_locationName
teams_locationName
<class 'str'>    2922
Name: count, dtype: int64
Column: teams_firstYearOfPlay
teams_firstYearOfPlay
<class 'str'>      2921
<class 'float'>       1
Name: count, dtype: int64
Column: teams_shortName
teams_shortName
<class 'str'>    2922
Name: count, dtype: int64
Column: teams_parentOrgName
teams_parentOrgName
<class 'str'>      2910
<class 'float'>      12
Name: count, dtype: int64
Column: teams_franchiseName
teams_franchiseName
<class 'str'>      2921
<class 'float'>       1
Name: count, dtype: int64
Column: teams_clubName
teams_clubName
<class 'str'>      2921
<class 'float'>       1
Name: co

In [20]:
# Example of normalizing a column that contains dictionaries
if 'some_column' in filtered_data.columns:
    normalized_df = pd.json_normalize(filtered_data['some_column'])
    filtered_data = pd.concat([filtered_data.drop(columns=['some_column']), normalized_df], axis=1)

In [35]:
# Filter for the target season and historical seasons
target_season = 2024
historical_seasons = [2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013]

# Filter the data for the target season and historical seasons
filtered_data = df[(df['teams_season'].isin(historical_seasons)) | (df['teams_season'] == target_season)].copy()

# Drop rows with NaN in 'teams_allStarStatus'
filtered_data = filtered_data.dropna(subset=['teams_allStarStatus'])

# Convert 'teams_allStarStatus' to binary (1 for 'Y', 0 for 'N' or any other value)
filtered_data.loc[:, 'allStarStatus'] = filtered_data['teams_allStarStatus'].apply(lambda x: 1 if x == 'Y' else 0)

# Features (X) and target (y)
X = filtered_data.drop(columns=['teams_allStarStatus', 'teams_season', 'league_abbreviation', 'allStarStatus'])
y = filtered_data['allStarStatus']

# Debugging: Check lengths of X and y
print(f"Length of X: {len(X)}")
print(f"Length of y: {len(y)}")

# --- Identify and normalize columns with dictionaries ---
for col in X.select_dtypes(include=['object']).columns:
    # Check if the column contains dictionaries
    if X[col].apply(type).eq(dict).any():
        print(f"Column '{col}' contains dictionaries. Normalizing...")
        try:
            # Normalize the column using json_normalize
            normalized_df = pd.json_normalize(X[col])
            # Add a prefix to avoid column name collisions
            normalized_df.columns = [f"{col}_{subcol}" for subcol in normalized_df.columns]
            # Concatenate the normalized data with the original DataFrame
            X = pd.concat([X.drop(columns=[col]), normalized_df], axis=1, ignore_index=False)
        except Exception as e:
            print(f"Error normalizing column '{col}': {e}")
    else:
        print(f"Column '{col}' does not contain dictionaries.")

# Encode categorical variables if necessary
X = pd.get_dummies(X, drop_first=True)

# Check for empty DataFrame before splitting
if not X.empty and not y.empty:
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize models
    models = {
        'LinearRegression': LinearRegression(),
        'LASSO': Lasso(),
        'Ridge': Ridge(),
        'DecisionTree': DecisionTreeRegressor(),
        'RandomForest': RandomForestRegressor()
    }

    # Store results
    results = {}

    # Train and evaluate each model
    for model_name, model in models.items():
        start_time = time.time()  # Start time for computation
        model.fit(X_train, y_train)  # Train the model
        y_pred = model.predict(X_test)  # Make predictions
        end_time = time.time()  # End time for computation

        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        computation_time = end_time - start_time

        # Store results
        results[model_name] = {
            'RMSE': rmse,
            'MSE': mse,
            'MAE': mae,
            'Computation Time (s)': computation_time
        }

    # Display results
    for model_name, metrics in results.items():
        print(f"Model: {model_name}")
        print(f"RMSE: {metrics['RMSE']:.4f}")
        print(f"MSE: {metrics['MSE']:.4f}")
        print(f"MAE: {metrics['MAE']:.4f}")
        print(f"Computation Time: {metrics['Computation Time (s)']:.4f}\n")
else:
    print("X or y is empty. Please check the data filtering process.")

Length of X: 2922
Length of y: 2922
Column 'teams_name' does not contain dictionaries.
Column 'teams_abbreviation' does not contain dictionaries.
Column 'teams_teamName' does not contain dictionaries.
Column 'teams_locationName' does not contain dictionaries.
Column 'teams_firstYearOfPlay' does not contain dictionaries.
Column 'teams_shortName' does not contain dictionaries.
Column 'teams_parentOrgName' does not contain dictionaries.
Column 'teams_franchiseName' does not contain dictionaries.
Column 'teams_clubName' does not contain dictionaries.
Column 'teams_springLeague' does not contain dictionaries.
Column 'teams_springVenue' does not contain dictionaries.
Column 'teams_venue_name' does not contain dictionaries.
Column 'teams_venue_link' does not contain dictionaries.
Column 'teams_sport_link' does not contain dictionaries.
Column 'teams_sport_name' does not contain dictionaries.
Column 'teams_league_name' does not contain dictionaries.
Column 'teams_league_link' does not contain 

ValueError: Found input variables with inconsistent numbers of samples: [5719, 2922]

In [37]:
def preprocess_data(df, target_season, historical_seasons):
    # Create a copy to avoid modifying the original dataframe
    filtered_data = df.copy()

    print(f"Initial dataframe shape: {filtered_data.shape}")

    # Filter for seasons
    filtered_data = filtered_data[
        (filtered_data['teams_season'].isin(historical_seasons)) |
        (filtered_data['teams_season'] == target_season)
    ]
    print(f"Shape after season filtering: {filtered_data.shape}")

    # Drop rows with NaN in 'teams_allStarStatus'
    filtered_data = filtered_data.dropna(subset=['teams_allStarStatus'])
    print(f"Shape after dropping NaN values: {filtered_data.shape}")

    # Convert 'teams_allStarStatus' to binary
    filtered_data['allStarStatus'] = filtered_data['teams_allStarStatus'].apply(lambda x: 1 if x == 'Y' else 0)

    # Create index to preserve row alignment
    original_index = filtered_data.index

    # Separate features and target
    columns_to_drop = ['teams_allStarStatus', 'teams_season', 'league_abbreviation', 'allStarStatus']
    features = filtered_data.columns.difference(columns_to_drop)

    X = filtered_data[features].copy()
    y = filtered_data['allStarStatus']

    print(f"\nFeature matrix shape (X): {X.shape}")
    print(f"Target vector shape (y): {y.shape}")

    # Process dictionary columns
    for col in X.select_dtypes(include=['object']).columns:
        print(f"\nProcessing column: {col}")

        # Check if column contains dictionaries
        sample_non_null = X[col].dropna().iloc[0] if not X[col].dropna().empty else None

        if isinstance(sample_non_null, dict):
            print(f"Normalizing dictionary column: {col}")
            try:
                # Fill NaN values with empty dictionaries
                X[col] = X[col].fillna({})

                # Normalize the column
                normalized_df = pd.json_normalize(X[col].tolist())

                # Add prefix to columns
                normalized_df.columns = [f"{col}_{subcol}" for subcol in normalized_df.columns]

                # Remove original column and add normalized columns
                X = X.drop(columns=[col])
                X = pd.concat([X, normalized_df], axis=1)

                print(f"Successfully normalized {col}")
            except Exception as e:
                print(f"Error normalizing {col}: {str(e)}")
                # If normalization fails, drop the column
                X = X.drop(columns=[col])

    # Handle categorical columns
    categorical_columns = X.select_dtypes(include=['object']).columns

    # Convert categorical columns to string type and fill NaN values
    for col in categorical_columns:
        X[col] = X[col].astype(str).fillna('missing')

    # Create dummy variables with a maximum number of categories
    X_encoded = pd.DataFrame(index=X.index)

    for col in categorical_columns:
        # Get value counts and keep only top N categories
        top_categories = X[col].value_counts().nlargest(10).index  # Adjust 10 as needed

        # Create dummies only for top categories
        dummies = pd.get_dummies(X[col].apply(lambda x: x if x in top_categories else 'other'),
                               prefix=col,
                               drop_first=True)

        X_encoded = pd.concat([X_encoded, dummies], axis=1)

    # Add non-categorical columns
    numeric_columns = X.select_dtypes(exclude=['object']).columns
    X_encoded = pd.concat([X_encoded, X[numeric_columns]], axis=1)

    # Ensure index alignment
    X_encoded = X_encoded.reindex(original_index)

    print(f"\nFinal shapes after preprocessing:")
    print(f"X shape: {X_encoded.shape}")
    print(f"y shape: {y.shape}")

    return X_encoded, y

def train_models(X, y):
    if X.shape[0] != y.shape[0]:
        raise ValueError(f"Feature matrix and target vector have different lengths: X={X.shape[0]}, y={y.shape[0]}")

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize models
    models = {
        'LinearRegression': LinearRegression(),
        'LASSO': Lasso(),
        'Ridge': Ridge(),
        'DecisionTree': DecisionTreeRegressor(),
        'RandomForest': RandomForestRegressor()
    }

    results = {}

    for model_name, model in models.items():
        print(f"\nTraining {model_name}...")
        start_time = time.time()

        try:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            mse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            computation_time = time.time() - start_time

            results[model_name] = {
                'RMSE': rmse,
                'MSE': mse,
                'MAE': mae,
                'Computation Time (s)': computation_time
            }

            print(f"{model_name} training completed successfully")
        except Exception as e:
            print(f"Error training {model_name}: {str(e)}")

    return results

# Usage
target_season = 2024
historical_seasons = [2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013]

# Process the data
X, y = preprocess_data(df, target_season, historical_seasons)

# Train and evaluate models
try:
    results = train_models(X, y)

    # Print results
    print("\nModel Performance:")
    for model_name, metrics in results.items():
        print(f"\n{model_name}:")
        for metric_name, value in metrics.items():
            print(f"{metric_name}: {value:.4f}")
except Exception as e:
    print(f"Error in model training: {str(e)}")

Initial dataframe shape: (15023, 121)
Shape after season filtering: (2922, 121)
Shape after dropping NaN values: (2922, 121)

Feature matrix shape (X): (2922, 118)
Target vector shape (y): (2922,)

Processing column: jerseyNumber

Processing column: league_active

Processing column: league_allStarDate

Processing column: league_conferencesInUse

Processing column: league_divisionsInUse

Processing column: league_firstDate2ndHalf

Processing column: league_gameLevelGamedayType

Processing column: league_hasPlayoffPoints

Processing column: league_hasSplitSeason

Processing column: league_hasWildCard

Processing column: league_lastDate1stHalf

Processing column: league_name

Processing column: league_nameShort

Processing column: league_offSeasonEndDate

Processing column: league_offseasonStartDate

Processing column: league_postSeasonEndDate

Processing column: league_postSeasonStartDate

Processing column: league_preSeasonEndDate

Processing column: league_preSeasonStartDate

Processin